In [2]:
import os
import random
import re

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer, WordNetLemmatizer
import spacy
from spacy.tokenizer import Tokenizer

import torch
import torch.nn as nn
import torchtext
from torchtext import data, legacy
from torchtext.vocab import Vectors, GloVe

In [11]:
train = pd.read_csv('../train_tweet.csv')
test = pd.read_csv('../test_tweets.csv')
print("TRAIN")
print(train.shape)
print(train.info())
print("\n=====================================\n")
print("TEST")
print(test.shape)
print(test.info())

TRAIN
(31962, 3)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31962 entries, 0 to 31961
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      31962 non-null  int64 
 1   label   31962 non-null  int64 
 2   tweet   31962 non-null  object
dtypes: int64(2), object(1)
memory usage: 749.2+ KB
None


TEST
(17197, 2)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17197 entries, 0 to 17196
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      17197 non-null  int64 
 1   tweet   17197 non-null  object
dtypes: int64(1), object(1)
memory usage: 268.8+ KB
None


In [12]:
train.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [15]:
test.head()

,id,tweet
0,31963,#studiolife #aislife #requires #passion #dedic...
1,31964,@user #white #supremacists want everyone to s...
2,31965,safe ways to heal your #acne!! #altwaystohe...
3,31966,is the hp and the cursed child book up for res...
4,31967,"3rd #bihday to my amazing, hilarious #nephew..."


In [21]:
print("TRAIN SAMPLES")
for i in range(10):
  print(train['tweet'][i])
  print("------------------")
print("\n=====================================\n")
print("TEST SAMPLES")
for i in range(10):
  print(test['tweet'][i])
  print("------------------")

TRAIN SAMPLES
 @user when a father is dysfunctional and is so selfish he drags his kids into his dysfunction.   #run
------------------
@user @user thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx.    #disapointed #getthanked
------------------
  bihday your majesty
------------------
#model   i love u take with u all the time in urð±!!! ððððð¦ð¦ð¦  
------------------
 factsguide: society now    #motivation
------------------
[2/2] huge fan fare and big talking before they leave. chaos and pay disputes when they get there. #allshowandnogo  
------------------
 @user camping tomorrow @user @user @user @user @user @user @user dannyâ¦
------------------
the next school year is the year for exams.ð¯ can't think about that ð­ #school #exams   #hate #imagine #actorslife #revolutionschool #girl
------------------
we won!!! love the land!!! #allin #cavs #champions #cleveland #clevelandcavaliers  â¦ 
------------------
 @user @user wel